# SINGLE SITE ANOMALY DETECTION AND CORRECTION
This script performs anomaly detection and correction for multiple sensors at a single monitoring site.
The script imports data, performs initial anomaly detection based on rules, uses models (ARIMA and 4 flavors of LSTM)
and associated thresholds to detect anomalies, aggregates for overall anomaly detection, and performs correction
based on ARIMA.

Site: Logan River at Main Street

Sensors: temperature, specific conductance, pH, dissolved oxygen

Created: Amber Jones, March 2021

## Import Libraries and Functions

In [ ]:
import pandas as pd
from PyHydroQC import anomaly_utilities
from PyHydroQC import model_workflow
from PyHydroQC import rules_detect
from PyHydroQC import ARIMA_correct
from PyHydroQC import modeling_utilities
from PyHydroQC.model_workflow import ModelType

## Import Parameters

In [ ]:
# Parameters may be specified in a parameters file or in this script
from Examples.FB_parameters import site_params, LSTM_params, calib_params

## Retrieve data
Creates an object with a data frame specific to each sensor.

In [ ]:
site = 'MainStreet'
sensors = ['temp', 'cond', 'ph', 'do']
sensor_array = anomaly_utilities.get_data(sensors, filename='MS2018.csv', path='LRO_data/')

## Rules Based Anomaly Detection
Performs checks for range and persistence. Min/max range and duration are defined in the parameters.
Data outside a range or persisting longer than a duration are detected as anomalous, corrected by linear interpolation.
The output is a column 'observed' of intermediate results that are used for subsequent modeling.

In [ ]:
range_count = dict()
persist_count = dict()
rules_metrics = dict()
for snsr in sensor_array:
    sensor_array[snsr], range_count[snsr] = \
        rules_detect.range_check(sensor_array[snsr], site_params[site][snsr]['max_range'], site_params[site][snsr]['min_range'])
    sensor_array[snsr], persist_count[snsr] = \
        rules_detect.persistence(sensor_array[snsr], site_params[site][snsr]['persist'], output_grp=True)
    sensor_array[snsr] = rules_detect.interpolate(sensor_array[snsr])
print('Rules based detection complete.\n')


### Detect Calibration Events
Calibration events are identified where persistence is within a certain window (e.g., after a sensor is returned to
the water, it is 'stuck' and reports the same values for several time steps), the time of day within a certain window,
and where these overlap for all sensors. When this occurs, an event is identified. Hours and durations are defined in
the parameters. A subset of sensors are selected (1:4) because temperature is not calibrated. Calibration events 
detected here should be reviewed, compared to field records, and organized as input to the following step: drift correction.

In [ ]:
calib_sensors = sensors[1:4]
input_array = dict()
for snsr in calib_sensors:
    input_array[snsr] = sensor_array[snsr]
all_calib, all_calib_dates, df_all_calib, calib_dates_overlap = \
    rules_detect.calib_overlap(calib_sensors, input_array, calib_params)


### Perform Linear Drift Correction
Drift correction is performed when data are shifted due to calibration or cleaning of a sensor. To perform drift
correction, the routine requires a start date, an end date, and a gap value to shift the data. This step requires
some manual effort either after calibration events are detected as above or by reviewing field records and raw data.
In this case, the inputs were determined from the scripts that technicians ran on these data.

In [ ]:
calib_dates = dict()
for cal_snsr in calib_sensors:
    calib_dates[cal_snsr] = \
        pd.read_csv('LRO_data/' + site + '_' + cal_snsr + '_calib_dates.csv', header=1, parse_dates=True, infer_datetime_format=True)
    calib_dates[cal_snsr]['start'] = pd.to_datetime(calib_dates[cal_snsr]['start'])
    calib_dates[cal_snsr]['end'] = pd.to_datetime(calib_dates[cal_snsr]['end'])
    calib_dates[cal_snsr] = calib_dates[cal_snsr].loc[(calib_dates[cal_snsr]['start'] > min(sensor_array[cal_snsr].index)) &
                                                      (calib_dates[cal_snsr]['start'] < max(sensor_array[cal_snsr].index))]

    for i in range(min(calib_dates[cal_snsr].index), max(calib_dates[cal_snsr].index)):
        result, sensor_array[cal_snsr]['observed'] = rules_detect.lin_drift_cor(
                                                        sensor_array[cal_snsr]['observed'],
                                                        calib_dates[cal_snsr]['start'][i],
                                                        calib_dates[cal_snsr]['end'][i],
                                                        calib_dates[cal_snsr]['gap'][i],
                                                        replace=True)


## Model Based Anomaly Detection
Generates 5 models. Each predict one step ahead using immediately previous data. Anomalies are detected by comparing
model predictions to observations. Dynamic thresholds are based on model variability. Settings for ARIMA, LSTM, and
threshold determination are set in the parameters. Metrics are output for each model type. Anomalies are
aggregated so that any detection results in an anomaly.

### ARIMA Detection
ARIMA models use a combination of past data in a linear form to predict the next value. These models are univariate.
Each ARIMA model requires the parameters p, d, q, which can be defined automatically as shown here.

In [ ]:
all_pdq = dict()
for snsr in sensors:
    all_pdq[snsr] = modeling_utilities.pdq(sensor_array[snsr]['observed'])
    print(snsr + ' (p, d, q) = ' + str(all_pdq[snsr]))
    site_params[site][snsr]['pdq'] = all_pdq[snsr]

ARIMA = dict()
for snsr in sensors:
    ARIMA[snsr] = model_workflow.ARIMA_detect(sensor_array[snsr], snsr, site_params[site][snsr],
                                              rules=False, plots=False, summary=False, compare=False)
print('ARIMA detection complete.\n')

### LSTM Detection
LSTM models create a neural network that uses a sequence of past values to make a prediction. LSTM parameters are
defined in the parameters file.


#### DATA: univariate, MODEL: vanilla
Uses a single sensor and several time steps prior to predict the next point.

In [ ]:
LSTM_univar = dict()
for snsr in sensors:
    LSTM_univar[snsr] = model_workflow.LSTM_detect_univar(
            sensor_array[snsr], snsr, site_params[site][snsr], LSTM_params, model_type=ModelType.VANILLA,
            rules=False, plots=False, summary=False, compare=False, model_output=False, model_save=False)

#### DATA: univariate,  MODEL: bidirectional
Uses a single sensor and data before and after to predict a point.

In [ ]:
LSTM_univar_bidir = dict()
for snsr in sensors:
    LSTM_univar_bidir[snsr] = model_workflow.LSTM_detect_univar(
            sensor_array[snsr], snsr, site_params[site][snsr], LSTM_params, model_type=ModelType.BIDIRECTIONAL,
            rules=False, plots=False, summary=False, compare=False, model_output=False, model_save=False)


#### DATA: multivariate,  MODEL: vanilla
Uses multiple sensors as inputs and outputs and data prior to predict the next point.

In [ ]:
LSTM_multivar = model_workflow.LSTM_detect_multivar(
        sensor_array, sensors, site_params[site], LSTM_params, model_type=ModelType.VANILLA,
        rules=False, plots=False, summary=False, compare=False, model_output=False, model_save=False)


#### DATA: multivariate,  MODEL: bidirectional
Uses multiple sensors as inputs and outputs and data before and after to predict a point.

In [ ]:
LSTM_multivar_bidir = model_workflow.LSTM_detect_multivar(
        sensor_array, sensors, site_params[site], LSTM_params, model_type=ModelType.BIDIRECTIONAL,
        rules=False, plots=False, summary=False, compare=False, model_output=False, model_save=False)


### Aggregate Detections for All Models
Aggregates the results from all models and outputs metrics.


In [ ]:
aggregate_results = dict()
for snsr in sensors:
    models = dict()
    models['ARIMA'] = ARIMA[snsr].df
    models['LSTM_univar'] = LSTM_univar[snsr].df_anomalies
    models['LSTM_univar_bidir'] = LSTM_univar_bidir[snsr].df_anomalies
    models['LSTM_multivar'] = LSTM_multivar.all_data[snsr]
    models['LSTM_multivar_bidir'] = LSTM_multivar_bidir.all_data[snsr]
    results_all = anomaly_utilities.aggregate_results(sensor_array[snsr], models, verbose=True, compare=True)
    aggregate_results[snsr] = results_all


## Correction
Correction is performed using piecewise ARIMA- small models determined for each period of anomalous data, along with
forecast and backcast, which consider data before and after a period of anomalous data and are merged together to ensure there is no dicontinuities.

In [ ]:
corrections = dict()
for snsr in sensors:
    corrections[snsr] = ARIMA_correct.generate_corrections(aggregate_results[snsr], 'observed', 'detected_event')


#########################################

